In [1]:
'''Hotpotqa 
Qn -> answer + context (CL < 600)'''

'Hotpotqa \nQn -> answer + context (CL < 600)'

In [2]:
from datasets import load_dataset
import pickle

/raid/ganesh/vishak/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
hotpotqa_dataset = load_dataset('hotpot_qa', 'distractor')
train_data = hotpotqa_dataset['train']

In [9]:
finetune_traindata_alpaca_format = []
for item in train_data:
    titles = item['supporting_facts']['title']
    context=''
    for title in titles:
        context_titles = item['context']['title']
        index = context_titles.index(title)
        context = context + ''.join(item['context']['sentences'][index])
    
    if (len(context) <= 600):
        temp={}
        instruction = instruction = "Generate attributable background context from Wikipedia to answer the given question and answer the given question based on the generated context."
        input_ = "Question: " + item['question']
        output = "###Answer: " + item['answer'] + " ###Context: " + context + " ###END"
        temp['instruction'] = instruction
        temp['input'] = input_
        temp['output'] = output
        finetune_traindata_alpaca_format.append(temp)

In [10]:
finetune_traindata_alpaca_format[0]

{'instruction': 'Generate attributable background context from Wikipedia to answer the given question and answer the given question based on the generated context.',
 'input': "Question: Which magazine was started first Arthur's Magazine or First for Women?",
 'output': '###Answer: Arthur\'s Magazine ###Context: Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book".First for Women is a woman\'s magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies. ###END'}

In [11]:
len(finetune_traindata_alpaca_format)

17881

In [12]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import pickle

dataset = Dataset.from_pandas(pd.DataFrame(data=finetune_traindata_alpaca_format))
dataset_dict = DatasetDict({"train": dataset})

dataset_dict.save_to_disk("../lora-instruct/finetune_hotpotqa_traindata_input_qn_op_answer_context_CL600")

Saving the dataset (1/1 shards): 100%|██████████| 17881/17881 [00:00<00:00, 1269351.26 examples/s]
